In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import glob
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2
import random
import sys
from tqdm import tqdm
from time import sleep

In [9]:
path_an = str(Path().absolute()) + "\\archive\\annotations"
img_base_path = str(Path().absolute()) + "\\archive\\images\\"

window_size = 40

In [10]:
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "name":[],    
            "file":[],
            "width":[],
            "height":[],
           }

for anno in glob.glob(path_an+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag:
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    
                    
        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('\\')[-1][0:-4] +".png"]
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]


In [11]:
data=pd.DataFrame(dataset)
img_base_path = str(Path().absolute()) + "\\archive\\images\\"

face_model = cv2.CascadeClassifier(str(Path().absolute()) + "\\haarcascade_frontalface_default.xml")

with tqdm(total=len(data.index), file=sys.stdout) as pbar:
    for index, row in data.iterrows():
        pbar.update(1)
        
        
        if row['xmax'] - row['xmin'] < 20:
            data = data.drop(index)
            continue
        
        
        image=cv2.imread(img_base_path + row['file'])
        cropped_image=image[row['ymin']:row['ymax'],row['xmin']:row['xmax']]
        
        """
        if len(face_model.detectMultiScale(cropped_image,scaleFactor=1.01, minNeighbors=4)) == 0:
            data = data.drop(index)
            continue
        """
        
        fm = cv2.Laplacian(cropped_image, cv2.CV_64F).var()

        if fm < 500:
            data = data.drop(index)
            continue
        
data

100%|█████████████████████████████████████████████████████████████████████████████| 4072/4072 [00:27<00:00, 149.11it/s]


,xmin,ymin,xmax,ymax,name,file,width,height
1,185,100,226,144,with_mask,maksssksksss0.png,512,366
2,325,90,360,141,without_mask,maksssksksss0.png,512,366
3,321,34,354,69,with_mask,maksssksksss1.png,400,156
4,224,38,261,73,with_mask,maksssksksss1.png,400,156
6,143,74,174,115,with_mask,maksssksksss1.png,400,156
...,...,...,...,...,...,...,...,...
4067,263,62,287,85,with_mask,maksssksksss98.png,400,267
4068,344,80,377,106,with_mask,maksssksksss98.png,400,267
4069,181,54,273,162,mask_weared_incorrect,maksssksksss99.png,400,267
4070,99,87,176,165,with_mask,maksssksksss99.png,400,267


In [12]:
negative_dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "name":[],    
            "file":[],
            "width":[],
            "height":[],
           }

old_name = ''

images = []

with tqdm(total=len(data.index), file=sys.stdout) as pbar:
    for index, row in data.iterrows():
        pbar.update(1)
        
        if old_name == row['file']:
            continue
        old_name = row['file']
        
        image=cv2.imread(img_base_path + row['file'])

        for i in range(100):
            x = random.randint(0,row['width']-window_size)
            y = random.randint(0,row['height']-window_size)
            if abs(x - row['xmin']) < max(window_size, row['xmax'] - row['xmin']):
                continue
            if abs(y - row['ymin']) < max(window_size, row['ymax'] - row['ymin']):
                continue
                
            negative_dataset['xmin'] += [x]
            negative_dataset['ymin'] += [y]
            negative_dataset['xmax'] += [x+window_size]
            negative_dataset['ymax'] += [y+window_size]
            negative_dataset['name'] += ['no_face']
            negative_dataset['file'] += [row['file']]
            negative_dataset['width'] += [row['width']]
            negative_dataset['height'] += [row['height']]
            break
pd.DataFrame(negative_dataset)
data = data.append(pd.DataFrame(negative_dataset))
data

100%|█████████████████████████████████████████████████████████████████████████████| 1978/1978 [00:07<00:00, 260.77it/s]


,xmin,ymin,xmax,ymax,name,file,width,height
1,185,100,226,144,with_mask,maksssksksss0.png,512,366
2,325,90,360,141,without_mask,maksssksksss0.png,512,366
3,321,34,354,69,with_mask,maksssksksss1.png,400,156
4,224,38,261,73,with_mask,maksssksksss1.png,400,156
6,143,74,174,115,with_mask,maksssksksss1.png,400,156
...,...,...,...,...,...,...,...,...
701,266,152,306,192,no_face,maksssksksss94.png,400,226
702,21,178,61,218,no_face,maksssksksss96.png,400,280
703,119,173,159,213,no_face,maksssksksss97.png,301,400
704,246,167,286,207,no_face,maksssksksss98.png,400,267


In [13]:
data['class']=data['name'].map({'no_face':-1, 'with_mask':0, 'mask_weared_incorrect':1, 'without_mask':2})

In [14]:
data.to_csv('data.csv',index=False)